<a href="https://colab.research.google.com/github/OJB-Quantum/Notebooks-for-Ideas/blob/main/Image_to_SVG_Converter_Attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Pillow scikit-image cairosvg

In [ ]:
from google.colab import files
from PIL import Image
import numpy as np
from skimage import measure
import io
from IPython.display import SVG, display

In [ ]:
def upload_image():
    uploaded = files.upload()  # Allows you to upload an image from your local machine
    for filename in uploaded.keys():
        return filename  # Return the filename of the uploaded file

def resize_image(image, max_resolution=(2560, 1440)):
    # Resize the image to fit within 2560x1440 while maintaining the aspect ratio
    image.thumbnail(max_resolution, Image.ANTIALIAS)
    return image

def convert_image_to_svg(image_path, output_svg_path):
    # Load the image and reduce its resolution
    image = Image.open(image_path)
    image = resize_image(image, max_resolution=(2560, 1440))  # Resize to 2.5K resolution

    # Convert the image to 8-bit (256 colors)
    image = image.convert('P', palette=Image.ADAPTIVE, colors=256)

    # Get the color palette (list of RGB values)
    palette = image.getpalette()
    palette = [palette[i:i+3] for i in range(0, len(palette), 3)]

    # Convert image to a NumPy array
    image_array = np.array(image)

    # Create an SVG file and write the contours with color fills
    svg_data = io.StringIO()
    svg_data.write('<svg xmlns="http://www.w3.org/2000/svg" version="1.1">\n')

    # Loop over all the colors in the palette
    unique_colors = np.unique(image_array)
    for color_index in unique_colors:
        # Extract contours for the current color
        mask = (image_array == color_index).astype(np.uint8)
        contours = measure.find_contours(mask, level=0.8)

        # Get the RGB value for the current color from the palette
        color_rgb = palette[color_index]
        fill_color = f'rgb({color_rgb[0]},{color_rgb[1]},{color_rgb[2]})'

        for contour in contours:
            svg_data.write(f'<path d="M ')
            for point in contour:
                svg_data.write(f'{point[1]},{point[0]} ')
            svg_data.write(f'" fill="{fill_color}" stroke="none"/>\n')

    svg_data.write('</svg>')

    # Save the SVG file
    with open(output_svg_path, 'w') as f:
        f.write(svg_data.getvalue())

    return svg_data.getvalue()

def display_svg(svg_content):
    # Convert the SVG content to a data URL for display
    svg_bytes = svg_content.encode('utf-8')
    display(SVG(data=svg_bytes))

# Upload an image
image_path = upload_image()

# Path to the output SVG file
output_svg_path = 'output_image.svg'

# Convert the image to SVG with 8-bit color representation and 2.5K resolution
svg_content = convert_image_to_svg(image_path, output_svg_path)

# Preview the SVG
display_svg(svg_content)

print(f"SVG saved at: {output_svg_path}")

In [ ]:
# Maximum allowed window size
MAX_WIDTH = 2560
MAX_HEIGHT = 1440

def upload_image():
    # Function to upload the image
    from google.colab import files
    uploaded = files.upload()  # Allows you to upload an image from your local machine
    for filename in uploaded.keys():
        return filename  # Return the filename of the uploaded file

def scale_dimensions(original_width, original_height):
    # Calculate scaling factor to fit within the 2560x1440 window
    width_scale = MAX_WIDTH / original_width
    height_scale = MAX_HEIGHT / original_height
    scale_factor = min(width_scale, height_scale)  # Use the smaller scale to fit both dimensions

    return scale_factor

def convert_image_to_svg(image_path, output_svg_path):
    # Load the image and quantize it to 8-bit (256 colors)
    image = Image.open(image_path).convert('P', palette=Image.ADAPTIVE, colors=256)

    # Get the original image dimensions
    original_width, original_height = image.size
    scale_factor = scale_dimensions(original_width, original_height)

    # Get the color palette (list of RGB values)
    palette = image.getpalette()
    palette = [palette[i:i+3] for i in range(0, len(palette), 3)]

    # Convert image to a NumPy array
    image_array = np.array(image)

    # Create an SVG file and write the contours with color fills
    svg_data = io.StringIO()
    svg_data.write(f'<svg xmlns="http://www.w3.org/2000/svg" version="1.1" width="{original_width * scale_factor}" height="{original_height * scale_factor}">\n')

    # Loop over all the colors in the palette
    unique_colors = np.unique(image_array)
    for color_index in unique_colors:
        # Extract contours for the current color
        mask = (image_array == color_index).astype(np.uint8)
        contours = measure.find_contours(mask, level=0.8)

        # Get the RGB value for the current color from the palette
        color_rgb = palette[color_index]
        fill_color = f'rgb({color_rgb[0]},{color_rgb[1]},{color_rgb[2]})'

        for contour in contours:
            svg_data.write(f'<path d="M ')
            for point in contour:
                # Apply scaling to the contour points
                svg_data.write(f'{point[1] * scale_factor},{point[0] * scale_factor} ')
            svg_data.write(f'" fill="{fill_color}" stroke="none"/>\n')

    svg_data.write('</svg>')

    # Save the SVG file
    with open(output_svg_path, 'w') as f:
        f.write(svg_data.getvalue())

    return svg_data.getvalue()

def display_svg(svg_content):
    # Convert the SVG content to a data URL for display
    svg_bytes = svg_content.encode('utf-8')
    display(SVG(data=svg_bytes))

# Upload an image
image_path = upload_image()

# Path to the output SVG file
output_svg_path = 'output_image.svg'

# Convert the image to SVG with 8-bit color representation and scaling
svg_content = convert_image_to_svg(image_path, output_svg_path)

# Preview the SVG
display_svg(svg_content)

print(f"SVG saved at: {output_svg_path}")